Libretas a utilizar

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score
from imblearn.over_sampling import SMOTE

## Procesamiento del dataset

In [ ]:
df_model = pd.read_csv('train.csv')

In [ ]:
df_original = pd.read_csv('train_dataset.csv')

In [ ]:
df_original['id'] = df_original.index

In [ ]:
df = pd.concat([df_model, df_original])

In [ ]:
df.set_index('id', inplace=True)

In [ ]:
df

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
id,,,,,,,,,,,,,,,,,,,,,
0,55,165,60,81.0,0.5,0.6,1,1,135,87,...,40,75,16.5,1,1.0,22,25,27,0,1
1,70,165,65,89.0,0.6,0.7,2,2,146,83,...,57,126,16.2,1,1.1,27,23,37,1,0
2,20,170,75,81.0,0.4,0.5,1,1,118,75,...,45,93,17.4,1,0.8,27,31,53,0,1
3,35,180,95,105.0,1.5,1.2,1,1,131,88,...,38,102,15.9,1,1.0,20,27,30,1,0
4,30,165,60,80.5,1.5,1.0,1,1,121,76,...,44,93,15.4,1,0.8,19,13,17,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38979,40,165,60,80.0,0.4,0.6,1,1,107,60,...,61,72,12.3,1,0.5,18,18,21,1,0
38980,45,155,55,75.0,1.5,1.2,1,1,126,72,...,76,131,12.5,2,0.6,23,11,12,0,0
38981,40,170,105,124.0,0.6,0.5,1,1,141,85,...,48,138,17.1,1,0.8,24,23,35,1,1


Al ser estas dos clases tan desbalanceadas, aparte de, no tener mucho que ver con algún sintoma a la hora de fumar, decidí eliminar ambas columnas

In [ ]:
df.drop(['hearing(right)', 'hearing(left)'], inplace=True, axis='columns')

In [ ]:
for column in df.columns.to_list():
  print(f'{column}: {df[column].nunique()}')

age: 18
height(cm): 15
weight(kg): 29
waist(cm): 548
eyesight(left): 20
eyesight(right): 18
systolic: 128
relaxation: 94
fasting blood sugar: 259
Cholesterol: 279
triglyceride: 393
HDL: 123
LDL: 286
hemoglobin: 144
Urine protein: 6
serum creatinine: 34
AST: 196
ALT: 230
Gtp: 444
dental caries: 2
smoking: 2


Separamos las columnas categoricas con las columnas numericas

In [ ]:
cat_columns = list(df[['Urine protein', 'dental caries']].columns)

In [ ]:
cat_columns

['Urine protein', 'dental caries']

In [ ]:
num_columns = [column for column in df.columns[:-1] if column not in cat_columns]

In [ ]:
num_columns

['age',
 'height(cm)',
 'weight(kg)',
 'waist(cm)',
 'eyesight(left)',
 'eyesight(right)',
 'systolic',
 'relaxation',
 'fasting blood sugar',
 'Cholesterol',
 'triglyceride',
 'HDL',
 'LDL',
 'hemoglobin',
 'serum creatinine',
 'AST',
 'ALT',
 'Gtp']

In [ ]:
df[num_columns].describe()

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,serum creatinine,AST,ALT,Gtp
count,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000,198240.000000
mean,44.271418,165.153375,66.906709,82.817163,1.007599,1.002519,122.301488,76.701085,98.547180,196.009988,127.445632,56.135951,114.700858,14.763003,0.891440,25.650847,26.667282,36.941455
std,11.886308,8.895587,12.656868,9.039363,0.422827,0.414241,12.920634,9.135720,16.514779,30.131156,67.330696,14.106668,31.601535,1.460424,0.188198,12.014495,21.119268,35.637177
min,20.000000,130.000000,30.000000,51.000000,0.100000,0.100000,71.000000,40.000000,46.000000,55.000000,8.000000,4.000000,1.000000,4.900000,0.100000,6.000000,1.000000,2.000000
25%,40.000000,160.000000,55.000000,77.000000,0.800000,0.800000,113.000000,70.000000,90.000000,175.000000,76.000000,46.000000,94.000000,13.800000,0.800000,20.000000,16.000000,18.000000
50%,40.000000,165.000000,65.000000,83.000000,1.000000,1.000000,121.000000,77.000000,96.000000,196.000000,113.000000,54.000000,114.000000,15.000000,0.900000,23.000000,22.000000,27.000000
75%,55.000000,170.000000,75.000000,89.000000,1.200000,1.200000,130.000000,82.000000,103.000000,217.000000,164.000000,65.000000,134.000000,15.800000,1.000000,29.000000,32.000000,44.000000
max,85.000000,190.000000,135.000000,129.000000,9.900000,9.900000,233.000000,146.000000,423.000000,445.000000,999.000000,359.000000,1860.000000,21.100000,11.600000,1090.000000,2914.000000,999.000000


Detectamos y eliminamos outliers

In [ ]:
for column in num_columns:
  up=df[column].mean() + 3*df[column].std()
  lw=df[column].mean() - 3*df[column].std()
  index= df[(df[column]>up)|(df[column]<lw)].index
  df.drop(index, inplace=True)

In [ ]:
smoking = df['smoking']
df.drop(['smoking'], inplace=True, axis='columns')

In [ ]:
for column in cat_columns:
  df[column] = LabelEncoder().fit_transform(df[column])

In [ ]:
df

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
id,,,,,,,,,,,,,,,,,,,,
4,30,165,60,80.5,1.5,1.0,121,76,91,155,87,44,93,15.4,0,0.8,19,13,17,0
7,55,155,60,84.5,0.7,0.9,137,91,100,282,165,51,198,14.5,0,0.7,16,15,16,0
8,40,165,70,89.0,0.7,1.0,130,80,104,243,163,59,150,15.7,0,0.9,24,21,31,0
9,40,155,50,73.0,1.5,1.5,105,70,64,183,27,55,122,13.2,0,0.7,22,16,14,0
10,40,155,50,68.0,0.8,0.6,107,60,71,165,76,48,101,12.9,2,0.7,17,11,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38978,40,170,65,77.0,1.5,1.5,110,62,91,187,84,79,91,16.1,0,0.9,28,43,36,1
38979,40,165,60,80.0,0.4,0.6,107,60,93,144,53,61,72,12.3,0,0.5,18,18,21,1
38980,45,155,55,75.0,1.5,1.2,126,72,91,227,100,76,131,12.5,1,0.6,23,11,12,0


In [ ]:
X = df

In [ ]:
y = smoking

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Balanceamos las clases para el entrenamiento de este, en este caso se usó SMOTE

In [ ]:
smote = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
xg_cl = xgb.XGBClassifier(objective='binary:logistic',max_depth=2, learning_rate=0.45, n_estimators=2048, subsample=0.75, seed=42)

In [ ]:
model = xg_cl.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)

In [ ]:
print(accuracy_score(y_test, preds))
print(roc_auc_score(y_test, preds))

0.77856933450601
0.7771766058739689


## Submission

Aquí ya procedemos a cargar el test, pasamos por el mismo preprocesamiento de el dataset original omitiendo la eliminación de outliers. Procedemos a crear nuestros predicts finales los cuales subiremos utilizando el modelo generado.

In [ ]:
df_test = pd.read_csv('test.csv').set_index('id')

In [ ]:
df_test.drop(['hearing(right)', 'hearing(left)'], inplace=True, axis='columns')

In [ ]:
df_test

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
id,,,,,,,,,,,,,,,,,,,,
159256,40,165,70,84.0,1.2,1.2,130,89,107,200,186,49,115,14.2,1,0.9,19,25,32,0
159257,80,160,60,93.0,1.0,1.0,144,72,93,172,158,35,104,13.0,1,1.1,20,12,24,0
159258,60,170,70,86.5,0.6,0.7,117,75,92,161,173,39,88,15.4,1,1.4,38,60,36,0
159259,40,160,50,67.0,0.3,0.4,116,62,91,213,47,75,128,14.5,1,0.6,25,18,10,1
159260,40,170,75,89.4,1.0,0.9,132,94,79,170,100,39,123,16.5,1,1.0,30,39,27,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265422,40,165,60,78.0,0.8,0.9,112,78,134,225,82,59,149,16.4,1,1.1,24,31,28,0
265423,40,170,85,95.0,1.2,1.2,130,90,97,180,186,44,100,16.0,2,1.0,25,31,38,0
265424,35,170,85,89.0,1.2,1.2,131,86,106,211,76,53,142,15.9,1,0.8,33,32,24,1


In [ ]:
for column in cat_columns:
  df_test[column] = LabelEncoder().fit_transform(df_test[column])

In [ ]:
preds = model.predict_proba(df_test)

In [ ]:
df_test.shape

(106171, 20)

In [ ]:
preds[:,1]

array([0.5853629 , 0.47313863, 0.32820383, ..., 0.42797807, 0.08579821,
       0.05365335], dtype=float32)

In [ ]:
df_test['smoking'] = preds[:,1]

In [ ]:
df_test.reset_index(inplace=True)

In [ ]:
submission = df_test[['id', 'smoking']]

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.shape

(106171, 2)